In [1]:
import pickle
import re
import string
import nltk
import pandas as pd
import spacy
import re
import string
from gensim import matutils, models
import scipy.sparse
from collections import Counter
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
spacy.prefer_gpu()
import scattertext as st
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [2]:
data_df = pd.read_pickle('./pickles/dtm_idf.pickle')
data_df

,aa,aaa,aaaaa,aaaaaaaa,aaaaaaaaaaaa,aaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaall,...,𝘽𝙊𝘽,𝘽𝙐𝙄𝙇𝘿𝙀𝙍,𝘾𝘼𝙉,𝙁𝙄𝙓,𝙃𝙀,𝙄𝙏,𝙏𝙃𝙀,𝙔𝙀𝙎,𝚃𝚑𝚒𝚜,𝚝𝚎𝚖𝚙𝚕𝚊𝚝𝚎
ama,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
askreddit,0.000464,0.000184,0.000000,0.000061,0.000052,0.00000,0.000061,0.000000,0.000061,0.00000,...,0.000123,0.000123,0.000123,0.000061,0.000123,0.000061,0.000123,0.000061,0.000000,0.000000
dankmemes,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
funny,0.000000,0.000000,0.000000,0.000000,0.000000,0.00139,0.000000,0.000000,0.000000,0.00139,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
memes,0.000000,0.000000,0.001659,0.000000,0.000000,0.00000,0.000000,0.013269,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001659,0.001659
science,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
worldnews,0.000519,0.000000,0.000000,0.000000,0.000259,0.00000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
wsb,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [3]:
def replace_row(text):
    return re.sub('a{3,}', '', text)


replaced = lambda x: replace_row(x)

In [4]:
data_replaced = pd.DataFrame(data_df.text.apply(replaced))

TypeError: expected string or bytes-like object

In [ ]:
corpus = st.CorpusFromPandas(data_replaced,
                             category_col='text',
                             text_col='text',
                             nlp=st.whitespace_nlp_with_sentences
                            ).build()

In [ ]:
html = st.produce_scattertext_explorer(
        corpus,
        category='democrat',
        category_name='Democratic',
        not_category_name='Republican',
        minimum_term_frequency=10,
        pmi_threshold_coefficient=5,
        width_in_pixels=1000,
        metadata=convention_df['speaker'],
        )

In [ ]:
data_replaced[-1:]

In [ ]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
# add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
#                   'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
# stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = TfidfVectorizer(ngram_range=(1,2))
data_cvn = cvn.fit_transform(data_replaced[-1:].text)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names()).T
# data_dtmn.columns = data_df.index
data_dtmn

In [ ]:
# data_dtmn['index'] = data_dtmn.index
# for i, row in enumerate(data_dtmn['index']):
#     candidate = re.match('a{2,}', str(row))
#     if candidate != None:
#         data_dtmn.drop(row, inplace=True)
#     if row == 'ababababba':
#         data_dtmn.drop(row, inplace=True) 
# data_dtmn.drop(columns=['index'], inplace=True)
# data_dtmn[20:30]

In [ ]:
data_dtmn.transpose()

In [ ]:
from gensim import matutils, models
import scipy.sparse

In [ ]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldan = models.LdaMulticore(corpus=corpusn, num_topics=50, id2word=id2wordn, passes=50)
ldan.print_topics()